__体素缺陷人工标注代码__

In [ ]:
import numpy as np
import scipy.io as io

data = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = data['defectdata']
data = np.array(data)
index = io.loadmat('../../Matlab_files/index.mat')
index = index['indexdata1']
index = np.array(index)
index_array = []
for i in range(len(index)):
    #提取一行进行处理
    deal_index = index[i]

    
    #增加维度，调整x、y、z列的顺序
    deal_index = deal_index[np.newaxis,:]
    deal_index = deal_index[:,[2, 0, 1]]

    #matlab中坐标转化到python中坐标
    new_x = 120 - deal_index[:,[0]]
    new_z = 120 - deal_index[:,[1]]
    new_y =  -deal_index[:,[2]]
    
    #将坐标存储到坐标列表中
    temp_array = [new_x, new_z, new_y]
    temp_array = np.squeeze(temp_array)
    index_array.append(temp_array)

index_array = np.array(index_array)
print(index_array)

for t in range(len(index_array)):
    embed_index = index_array[t]
    print(embed_index)
    x = embed_index[0]
    z = embed_index[1]
    y = embed_index[2]
    data[x][z][y] = 1

io.savemat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat',{'defectdata':data})


__数据集切割代码__

In [ ]:
import numpy as np
import scipy.io as io
import os

def cutmodule(matdata,saveinputindex):
    data = matdata['data']
    print(data.shape)
    nelx, nely, nelz = 120, 120, 120
    if (nelx - data.shape[0])%2 !=0:
        data = np.pad(data, ((1, 0), (0, 0), (0 ,0)))
    if (nelz - data.shape[1])%2 !=0:
        data = np.pad(data, ((0, 0), (1, 0), (0 ,0)))
    if (nely - data.shape[2])%2 !=0:
        data = np.pad(data, ((0, 0), (0, 0), (1 ,0)))
    data = np.pad(data, (((nelx - data.shape[0])//2, (nelx - data.shape[0])//2),
                    ((nely - data.shape[1])//2, (nely - data.shape[1])//2),
                    ((nelz - data.shape[2])//2, (nelx - data.shape[2])//2)))
    io.save('')
    tempdata = np.zeros((20, 20, 20))
    for k in range(0, data.shape[2], 20):               #沿y轴切割
        for j in range(0, data.shape[1], 20):           #沿z轴切割
            for i in range(0, data.shape[0], 20):       #沿x轴切割
                global m
                m += 1
                tempdata = data[i:i+20, j:j+20, k:k+20]
                # np.set_printoptions(threshold=np.inf)
                np.save(os.path.join(saveinputindex,'%d.npy'%m),tempdata)
                print('第：',m,'次')
                # print(tempdata)
                tempdata = np.zeros((20, 20, 20))

__非零矩阵标号检测__

In [ ]:
import numpy as np
import os
import glob

def nozerosdetect(data_index):

    global m
    n = 0
    zeros_array = []
    array = np.arange((m-215), (m+1))

    for i in range((m-215),(m+1)):
        path = os.path.join(data_index, str(i)+'.npy')
        data = np.load(path)
        if np.all(data == 0):
            n += 1
            zeros_array.append(i)
        else:
            zeros_array.append(0)

    print('一共',n,'个全零矩阵')

    zeros_array = np.array(zeros_array)
    nozeros_array = array - zeros_array
    np.save(os.path.join(data_index, 'detectmatrix.npy'), nozeros_array)

__主函数__

In [ ]:
import numpy as np
import os
import scipy.io as io
import glob

folder_address = '../../Matlab_files/dataset/originalmatdata/dataset/input/'
mat_address = '../../Matlab_files/dataset/originalmatdata/matdata/input/'

matdata_index = glob.glob(os.path.join(mat_address,'*.mat')) 

m = 0
for t in range(1,len(matdata_index)+1):

    print('读取第',t,'个数据')
    os.makedirs(os.path.join(folder_address,str(t)))
    path = os.path.join(mat_address,str(t)+'.mat')      #按顺序读取输入数据
    matdata = io.loadmat(path)
    saveinputindex = os.path.join(folder_address,str(t))
    cutmodule(matdata, saveinputindex)
    print('*'*30)
    data_index = os.path.join('../../Matlab_files/dataset/originalmatdata/dataset/input/',str(t)+'/')
    nozerosdetect(data_index)


__Mask转One-hot__

In [ ]:
import torch
index = torch.randint(7 ,size=(1, 1, 7, 7)).long()
print(index)
x_onehot = torch.zeros(1, 7, 7, 7).long()
x_onehot.scatter_(1, index, 1).float()
print(x_onehot.numpy())

In [ ]:
import torch
input = torch.randint(-5, 4, (2, 5))
print(type(input))
output = torch.zeros(2, 5)
index = torch.tensor([[3, 1, 2, 0], [1, 2, 0, 3]])
print(type(index))
output = output.scatter_(1, index, input)
print(output)


In [ ]:
import torch

src = torch.tensor([[0.3304, 0.5643, 0.2362, 0.1929, 0.2400, 0.6672, 0.5217, 0.4471],
        [0.0433, 0.2996, 0.9913, 0.4336, 0.8540, 0.8522, 0.0408, 0.1014]])

index = torch.tensor([[0, 1, 2, 0, 0, 1, 1, 2],
        [2, 0, 0, 1, 2, 1, 1, 1]])
input = torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

output = input.scatter(0, index ,src)
print(output)

__defectdata实现Mask转One-hot__

In [28]:
import scipy.io as io
import numpy as np
import torch
matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = matdata['defectdata']
np.set_printoptions(threshold=np.inf)
# print(data)
data = data[np.newaxis,np.newaxis,:,:,:]
data = torch.from_numpy(data).long()
print(data.shape)
np.set_printoptions(threshold=np.inf)
output = torch.zeros(1, 3, 120, 120, 120).long()            #output的channel数一定要等于类别数，同时也要等于mask中的类别数，因为scatter相当于是Mask
                                                            #拆成了与类别数相对应的One-hot，One-hot每个通道对应一个类别，总之：
                                                            #One-hot channel数 = 类别数 = mask中数字类别个数
print(output.shape)
output = output.scatter(1, data, 1)
print(output)

torch.Size([1, 1, 120, 120, 120])
torch.Size([1, 3, 120, 120, 120])
tensor([[[[[1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           ...,
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1]],

          [[1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           ...,
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1]],

          [[1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           ...,
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1]],

          ...,

          [[1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1, 1],
           ...,
           [1, 1, 1,  ..., 1, 1, 1],
           [1, 1, 1,  ..., 1, 1,

In [56]:
import scipy.io as io
import numpy as np

# matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
matdata = io.loadmat('./test.mat')
data = matdata['data']
print(data.shape)
b = np.nonzero(data)
b = np.array(b)
print(b)
print(type(b))

for i in range(b.shape[1]):
    print('第',i,'个数据')
    print(data[(b[0])[i]][(b[1])[i]][(b[2])[i]])
    m = i
print(m)

(120, 120, 120)
[[33 33 34 34 40 41 41 41 41 41 42 42 42 42 42 42 43 43 43 43 43 43 66 66
  66 66 67 67 67 67 68 68 68 68 68 68 69 69 69 69 69 69 69 69 70 70 70 70
  70 70 70 70 70 70 70 70 71 71 71 71 71 71 71 71 71 71 71 71 72 72 72 72
  72 72 72 72 73 73 73 73 76 76 77 77 78 78 79 79 87 87 87 87 88 88 88 88
  88 88 89 89 89 89 89 89 89 89]
 [59 60 59 60 60 58 58 60 61 61 57 58 58 61 61 62 57 57 58 61 62 62 52 52
  67 67 52 52 67 67 52 53 53 66 66 67 53 54 54 54 65 65 65 66 55 55 55 56
  58 59 60 61 63 64 64 64 56 56 57 57 58 59 60 61 62 62 63 63 57 58 58 59
  60 61 61 62 59 59 60 60 59 60 59 60 59 60 59 60 59 59 60 60 59 59 59 60
  60 60 58 59 59 59 60 60 60 61]
 [69 69 68 68 63 62 63 63 62 63 62 61 62 61 62 62 60 61 61 61 60 61 54 55
  54 55 54 55 54 55 56 57 58 57 58 56 58 59 60 61 59 60 61 58 61 62 63 64
  69 69 69 69 64 61 62 63 64 65 66 67 69 69 69 69 66 67 64 65 67 68 69 69
  69 68 69 67 68 69 68 69 67 67 66 66 66 66 65 65 67 69 67 69 67 68 69 67
  68 69 69 67 68 69 67 68 69 6

In [50]:
import torch

index = torch.randint(3, (1, 1, 5, 5)).long()
print('index:\n',index)

ouput_onehot = torch.zeros(1, 3, 5, 5).long()
ouput_onehot = ouput_onehot.scatter(1, index, 1)
print('output_onehot:\n',ouput_onehot)


index:
 tensor([[[[1, 1, 2, 0, 2],
          [0, 0, 2, 2, 0],
          [2, 1, 2, 2, 2],
          [0, 1, 0, 2, 0],
          [0, 1, 0, 2, 0]]]])
output_onehot:
 tensor([[[[0, 0, 0, 1, 0],
          [1, 1, 0, 0, 1],
          [0, 0, 0, 0, 0],
          [1, 0, 1, 0, 1],
          [1, 0, 1, 0, 1]],

         [[1, 1, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 1, 0, 0, 0],
          [0, 1, 0, 0, 0],
          [0, 1, 0, 0, 0]],

         [[0, 0, 1, 0, 1],
          [0, 0, 1, 1, 0],
          [1, 0, 1, 1, 1],
          [0, 0, 0, 1, 0],
          [0, 0, 0, 1, 0]]]])


In [57]:
import scipy.io as io
import numpy as np

matdata = io.loadmat('../../Matlab_files/top3d_Model/top3d_Model_1/iter-100/iters-100-006/defecttopModel_1_100_006.mat')
data = matdata['defectdata']
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            if data[i][j][k] != 0:
                data[i][j][k] = 1
            
io.savemat('test.mat',{'data':data})

In [63]:
import numpy as np
import scipy.io as io
import torch
matdata = io.loadmat('./test.mat')
data = matdata['data']
data = data[np.newaxis,np.newaxis,:,:,:]
data = torch.from_numpy(data).long()

print(data.shape)

output_onehot = torch.zeros(1, 2, 120, 120, 120)
output_onehot = output_onehot.scatter(1 ,data, 1)
print(output_onehot)

torch.Size([1, 1, 120, 120, 120])
tensor([[[[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],

          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],

          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],

          ...,

          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1.